In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
# 閾値を格納するリスト, 小さいほど当たりが出やすい
# エージェントはbanditsを知らない
bandits = [0.2, -0.5, -0.2, 0]
num_bandits = len(bandits)

In [3]:
# 当たりかハズレかを返す関数
def pullBandits(bandit):
    # 正規分布で乱数を発生させる
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [4]:
tf.reset_default_graph()

In [5]:
with tf.name_scope("weights"):
    # 1で初期化
    weights = tf.Variable(tf.ones([num_bandits]))
    tf.summary.histogram("weights", weights)

with tf.name_scope("chosen_action"):
    # 何番目のアームを引くか
    # weightsが大きいアームを引く
    chosen_action = tf.argmax(weights, 0)

with tf.name_scope("reward_holder"):
    # 報酬を保持
    reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)

with tf.name_scope("action_holder"):
    # aを保持
    action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

with tf.name_scope("responsible_weight"):
    # 選択したアームの重みを取り出す
    responsible_weight = tf.slice(weights, action_holder, [1])

with tf.name_scope("loss"):
    loss = -(tf.log(responsible_weight) * reward_holder)

with tf.name_scope("optimize"):
    update = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

In [6]:
"""ハイパーパラメータ"""
total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.1

In [7]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs", sess.graph)
    i = 0
    with tqdm() as pbar:
        while i < total_episodes:
            # εグリーディー(ランダムな手を打たせる）法
            if np.random.rand(1) < e:
                action = np.random.randint(num_bandits)
            else:
                action = sess.run(chosen_action)

            # 報酬を取得
            reward = pullBandits(bandits[action])

            _, resp, ww, _merged = sess.run(
                [update, responsible_weight, weights, merged],
                feed_dict={reward_holder: [reward], action_holder: [action]},
            )
            
            writer.add_summary(_merged)
            
            # 報酬の累積和の更新
            total_reward[action] += reward

            # 50回に１度，報酬を表示
            if i % 50 == 0:
                print("{}番目\tリワード・報酬の一覧: {}".format(i, total_reward))

            pbar.update(1)
            i += 1
            
        writer.close()

print("エージェントが考える最適なアームは，{}番目のアームです．".format(np.argmax(ww) + 1))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

0番目	リワード・報酬の一覧: [1. 0. 0. 0.]
50番目	リワード・報酬の一覧: [-3. 25.  0. -1.]
100番目	リワード・報酬の一覧: [-2. 35.  0. -2.]
150番目	リワード・報酬の一覧: [-3. 53.  2. -1.]
200番目	リワード・報酬の一覧: [-4. 76.  2. -1.]
250番目	リワード・報酬の一覧: [-4. 91.  3. -1.]
300番目	リワード・報酬の一覧: [ -4. 106.   2.  -3.]
350番目	リワード・報酬の一覧: [ -4. 137.   1.  -3.]
400番目	リワード・報酬の一覧: [ -6. 144.   1.  -2.]
450番目	リワード・報酬の一覧: [ -4. 151.   2.  -2.]
500番目	リワード・報酬の一覧: [ -3. 164.   3.  -3.]
550番目	リワード・報酬の一覧: [ -3. 179.   3.  -2.]
600番目	リワード・報酬の一覧: [ -3. 196.   4.  -2.]
650番目	リワード・報酬の一覧: [ -2. 226.   5.  -2.]
700番目	リワード・報酬の一覧: [ -1. 230.   5.  -1.]
750番目	リワード・報酬の一覧: [ -3. 248.   4.  -2.]
800番目	リワード・報酬の一覧: [ -4. 264.   5.  -2.]
850番目	リワード・報酬の一覧: [ -7. 282.   4.  -2.]
900番目	リワード・報酬の一覧: [ -7. 308.   4.  -2.]
950番目	リワード・報酬の一覧: [ -5. 329.   4.  -1.]

エージェントが考える最適なアームは，2番目のアームです．
